In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential

## 0. QuickStart
 - Keras Sequential
 - Custom Models & Training

In [3]:
tf.__version__

'2.7.0'

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 4s 0us/step


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

2022-02-28 18:15:20.435738: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.2557526 ,  0.25448284, -0.68653286,  0.5278821 , -0.383748  ,
        -0.75516635,  0.760613  ,  0.24992561, -0.17488328, -0.41297197]],
      dtype=float32)

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.07489607, 0.12475313, 0.04868258, 0.16397864, 0.06589787,
        0.0454534 , 0.20694791, 0.12418592, 0.08120451, 0.06399994]],
      dtype=float32)

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:
loss_fn(y_train[:1], predictions).numpy()

3.0910676

In [12]:
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2918 - accuracy: 0.9172
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1417 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1068 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0885 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0742 - accuracy: 0.9764


In [14]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0810 - accuracy: 0.9755 - 1s/epoch - 4ms/step


[0.08104712516069412, 0.9754999876022339]

In [15]:
probability_model = tf.keras.Sequential([
     model,
     tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.4914799e-08, 4.4651891e-09, 1.6479510e-06, 6.5585104e-05,
        1.8200306e-11, 1.3172630e-07, 3.5060015e-14, 9.9992859e-01,
        3.5378109e-09, 4.1059116e-06],
       [1.0836957e-08, 4.0089450e-05, 9.9995029e-01, 8.5035408e-06,
        1.3958228e-16, 6.6720133e-07, 9.6488954e-08, 1.1112435e-12,
        3.3972148e-07, 6.0494865e-13],
       [6.1206271e-07, 9.9926752e-01, 1.0001302e-04, 2.2819509e-06,
        2.0513035e-05, 1.7574204e-06, 4.0947148e-06, 5.8706576e-04,
        1.5468981e-05, 7.4225676e-07],
       [9.9998903e-01, 1.2606984e-10, 3.4378602e-06, 3.1354688e-08,
        3.5598005e-09, 9.6511337e-07, 5.6404274e-06, 9.5843245e-07,
        2.1977316e-09, 4.0883769e-08],
       [2.0385884e-05, 5.9416525e-09, 2.5071824e-06, 4.5842150e-08,
        9.9601197e-01, 8.4258221e-07, 1.0507313e-05, 2.1143665e-04,
        3.3278914e-06, 3.7388687e-03]], dtype=float32)>

- Custom Model & Training

In [54]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [55]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [56]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [57]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)


model = MyModel()

In [58]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [59]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [60]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [61]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [62]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )
    

Epoch 1, Loss: 0.13039414584636688, Accuracy: 96.12667083740234, Test Loss: 0.06028871238231659, Test Accuracy: 97.97000122070312
Epoch 2, Loss: 0.04161013662815094, Accuracy: 98.68500518798828, Test Loss: 0.06192103028297424, Test Accuracy: 98.0
Epoch 3, Loss: 0.022370316088199615, Accuracy: 99.28333282470703, Test Loss: 0.06451340764760971, Test Accuracy: 98.06999969482422
Epoch 4, Loss: 0.012508313171565533, Accuracy: 99.57666015625, Test Loss: 0.06398255378007889, Test Accuracy: 98.22000122070312
Epoch 5, Loss: 0.007958442904055119, Accuracy: 99.74333190917969, Test Loss: 0.06433290988206863, Test Accuracy: 98.43999481201172
